# Data from mutiple sources

## Intro

Now that we have a handle on using Folium and entering data from our own source, we are going to start looking at incorporating data from multiple locations to produce a richer map. We are going to look at a few different areas we can draw data from, and how we can incorporate them into our maps:
- [GeoJSON](https://geojson.org/) files. In this case we will be looking at two things to do with coastlines:
    - Whether our paths cross them
    - Whether the ship is close enough to them to have made a sighting
- Weather data - and how to display it
- Tide data - (if I can find a good source) and how to display it

## Aims 

- Defining functions we can then use later


## Data

Simply Googling is an effective way to find access to GeoJson and shapefile data. Searching for a "world map shapefile", for example, is liley to lead you to [World Bank](https://datacatalog.worldbank.org/search/dataset/0038272/World-Bank-Official-Boundaries) data. From here, we can find a link to a datasource called "World coastlines", which we should download. 

Note that, as with many types of data, some areas of the world are better covered than others. You may also find data behind a pay or register wall, or with other restrictions. There are, however, plently of data sources - feel free to have a go at searching for your own. For the examples in this notebook, we are going to be using the coastline data from World Bank. 


## Loading required libraries

As with the previous notebooks, we need to load some libraries. Some of these are going to be familiar, but we are also going to be using some new ones - these will be explained as we go along. Just like last time, we're going to install them using pip - this ensure that you've got them if you are coming to this notebook first, and if you've already got them, it won't do any harm.

In [ ]:
%pip install geopy
%pip install folium
%pip install shapely
import folium
import geopy
from geopy.distance import geodesic
from datetime import datetime, timedelta
import folium.plugins as plugins
import json
import shapely
import math

## Creating a new map

We are going to start by running a similar process to the previous notebook - creating a map, and adding a path to it. In this case, we are going to map only the UK (and  bit of the Northern French coast), and show the route taken by the ships in the sea shanty ["Spanish Ladies"](https://en.wikipedia.org/wiki/Spanish_Ladies). As we've prevosly done this, we are going to run through this in one cell with comments. 

Note: this time, we aren't going to worry about dates, as a: the Shanty doesn't actually specify a date, and b: from the previous notebook, we know that animations aren't working well with dates pre 1970. 

In [ ]:
## Create a map of the south coast of the UK, with enough padding to see the Isles of Scilly and Northern France, and room to add a legend

uk_coast = folium.Map(
    location=[50.5, -2.5],
    zoom_start=7,
    tiles='Stamen Terrain'
)

## The waypoints mentioned in Spanish Ladies

spanish_ladies = [
    {
        "current_name": "Ushant",
        "song_name": "Ushant",
        "lat": 48.5,
        "lon": -5.0
    },
    {
        "current_name": "Scilly Isles",
        "song_name": "Scilly",
        "lat": 49.9,
        "lon": -6.3
    },
    {
        "current_name": "Dodman",
        "song_name": "The Deadman",
        "lat": 50.2,
        "lon": -4.8
    },
    {
        "current_name": "Rame Head",
        "song_name": "Ram Head",
        "lat": 50.3,
        "lon": -4.2
    },
    {
        "current_name": "Start Point",
        "song_name": "Start",
        "lat": 50.2,
        "lon": -3.7
    },
    {
        "current_name": "Portland Bill",
        "song_name": "Portlant",
        "lat": 50.5,
        "lon": -2.5
    },
    {
        "current_name": "Cowes",
        "song_name": "The Wight",
        "lat": 50.8,
        "lon": -1.3
    },
    {
        "current_name": "Beachy Head",
        "song_name": "Beachy",
        "lat": 50.7,
        "lon": 0.2
    },
    {
        "current_name": "Fairlight",
        "song_name": "Fairly",
        "lat": 50.9,
        "lon": 0.6
    },
    {
        "current_name": "Dungeness",
        "song_name": "Dungeness",
        "lat": 50.9,
        "lon": 0.9
    },
    {
        "current_name": "South Foreland",
        "song_name": "The South Foreland Light",
        "lat": 51.1,
        "lon": 1.4
    },
    {
        "current_name": "The Downs",
        "song_name": "The Downs",
        "lat": 51.2,
        "lon": 1.6
    }
]

## Add the waypoints to the map

for waypoint in spanish_ladies:
    folium.Marker(
        location=[waypoint["lat"], waypoint["lon"]],
        popup=waypoint["current_name"],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(uk_coast)

## Add the path to the map

folium.PolyLine(
    locations=[[waypoint["lat"], waypoint["lon"]] for waypoint in spanish_ladies], ## Note: this is a shorthand to loop through a list
    color='red'
).add_to(uk_coast)

## Display the map

uk_coast


## Investigating the map

We can see that this map is a bit more interesting than the previous one - we have a path, and we have a coastline. We can also see that the path crosses the coastline - a bit of a problem for a ship! We can also see that the ships had a fairly long leg between Ushant and Scilly - are we sure they could see land the whole time? Finally, there are some strong tides, currents, and often weather in the English Channel - how can we show this on the map?

## Coastline data

The first area we are going to import is finding overlaps between our path and the coastline. For this, we need to use the coastline data we downloaded earlier. We are going to start by investigating this data file, as it covers the world, and we only need western Europe. Then, we are going to import it into the map and see how well it matches up with our map. From there, we can start to look at how we can use it to find overlaps.

## Investigating the GeoJSON file

If we open the Coastline GeoJSON file, we see that it is a huge file. We clearly don't need all of this, so we are going to look at how we can extract the data we need.

To start, we need to be a bit more confident on what we have. We can see that this is a list of lists of waypoints, and that some of the lists of waypoints are very long. We can also see that each of the lists of waypoints has an ID number, and that the first and last waypoint are the same. The maximum ID number is 4125, which as a gusstimate seems about right for every major continent and island in the world. We can also see that the very last coastline has the name "Null Island" - this is a common name for the point at 0,0 on a map, and here the list of waypoints wraps around 0,0 closely. 

To find the lists of waypoints we want, we can look through each of these lists, and see whether they're within a range we specify. 

In [ ]:
## Adding the coastlines to the map

## Load the coastline data

coastline_data = json.load(open("WB_Coastlines.geojson"))

## Add the coastline data to the map

folium.GeoJson(
    coastline_data,
    name='geojson'
).add_to(uk_coast)

## Display the map

uk_coast

In [ ]:
## Range of coordinates for Western Europe

western_europe_coords = {
    "lat": {
        "min": 47.0,
        "max": 57.0
    },
    "lon": {
        "min": -10.0,
        "max": 10.0
    }
}

## An empty list to store the coordinates of the coastline in Western Europe

west_europe_coastline = []

## load the json data

coastline_data = json.load(open("WB_Coastlines.geojson"))

## Loop through the features in the coastline data, looking for lists that contain locations in the western europe coordinates

for feature in coastline_data["features"]:
    #print(feature)
    if [coord[0] > western_europe_coords["lon"]["min"] and coord[0] < western_europe_coords["lon"]["max"] and coord[1] > western_europe_coords["lat"]["min"] and coord[1] < western_europe_coords["lat"]["max"] for coord in feature["geometry"]["coordinates"]]:
        west_europe_coastline.append(feature)
        continue
    #if feature["geometry"]["type"] == "LineString":
    #    for coord in feature["geometry"]["coordinates"]:
    #        #print(coord)
    #        if coord[0] > western_europe_coords["lon"]["min"] and coord[0] < western_europe_coords["lon"]["max"] and coord[1] > western_europe_coords["lat"]["min"] and coord[1] < western_europe_coords["lat"]["max"]:
    #            west_europe_coastline.append(feature)
    #            continue

for feature in west_europe_coastline:
    print(feature)

Then, to make sure this has worked correctly, we can plot the points we've found on a map and see what it looks like. This also gives us an oppertunity to look at how we can use the data we've found to find overlaps with our path by drawing the coastline on the map.

In [ ]:
## print the coordinates

#for coord in west_europe_coastline:
#    print(coord)

    ## Create a map of the south coast of the UK, with some enough padding to show Northern France and the Isles of Scilly and additional room.

uk_south_coast = folium.Map(
    location=[50.5, -1.5],
    zoom_start=7,
    tiles='Stamen Terrain'
)

## Add the western european coastline to the map

folium.GeoJson(
    coastline_data,
    name='geojson'
).add_to(uk_south_coast)

## Display the map

uk_south_coast

## Finding overlaps (writing your own functions)

We are going to be defining our own function to find overlaps between our path and the coastline. Having this function makes it much easier to seperate out the legs of our ships journey and check each one individually.

For this funciton we are going to use the [Shapely](https://shapely.readthedocs.io/en/stable/manual.html) library. This is a library that allows us to do geometric operations on shapes. In this case, we are going to be using it to find the intersection between our path and the coastline. We can see the documentation for the intersection function [here](https://shapely.readthedocs.io/en/stable/reference/shapely.intersection.html). From this documentation, we can see that we need to pass in two shapes, and the funciton will return the coordiates where they intersect. As we don't actually care about the coordinates, we can just pass in the shapes and check if the result is empty or not.

In [ ]:
## The function below takes a geojson dataset, and a pair of coordinates
## and checks whether the line defined by the coordinates intersects with
## any of the lines in the geojson dataset

def check_line_intersect(geojson, coords):
    for feature in geojson["features"]:
        if feature['geometry']['type'] == "LineString":
                if shapely.intersection(shapely.geometry.LineString(coords), shapely.geometry.LineString(feature['geometry']['coordinates'])):
                    return True
        elif feature['geometry']['type'] == "MultiLineString":
            for line in feature['geometry']['coordinates']:
                if shapely.intersection(shapely.geometry.LineString(coords), shapely.geometry.LineString(line)):
                    return True
    return False

## We loop through each pair of coords in the Spanish Ladies waypoints
## and use the function we just made to check whether the line between
## the current waypoint and the next waypoint intersects with the
## western european coastline

for i in range(len(spanish_ladies)-1):
    if check_line_intersect(coastline_data, [[spanish_ladies[i]["lon"], spanish_ladies[i]["lat"]], [spanish_ladies[i+1]["lon"], spanish_ladies[i+1]["lat"]]]):
        print("Line between {} and {} intersects with the coastline".format(spanish_ladies[i]["current_name"], spanish_ladies[i+1]["current_name"]))
    else:
        print("Line between {} and {} does not intersect with the coastline".format(spanish_ladies[i]["current_name"], spanish_ladies[i+1]["current_name"]))



## Making the overlaps show up on the map

Now we're able to tell which legs overlap, lets go back and adjust the data we're mapping to show this. Where we looped through and created a feature for each leg, we can now add a check to see if the leg overlaps the coastline. If it does, we can are going to change the colour of the line to red for that leg. 

In [ ]:
## For this, we will create a new map

uk_south_coast_showing_intersections = folium.Map(
    location=[50.5, -1.5],
    zoom_start=7,
    tiles='Stamen Terrain'
)

## Adding the waypoints can be just the same as before, they aren't going to change

for waypoint in spanish_ladies:
    folium.Marker(
        location=[waypoint["lat"], waypoint["lon"]],
        popup=waypoint["current_name"],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(uk_south_coast_showing_intersections)

## For this version of the map, we'll add the coastline data to the map
## This will be useful as the coastline data doesn't neccessarily follow the 
## coast shown on the map tiles exactly - by mapping it we can visually double check
## This can be just as before

folium.GeoJson(
    coastline_data,
    name='geojson'
).add_to(uk_south_coast_showing_intersections)

## Now we need to add the lines between the waypoints
## This is where things are going to be a bit different - we need to check whether
## the line between each pair of waypoints intersects with the coastline
## If it does, we'll draw it in red, if it doesn't, we'll draw it in green

for i in range(len(spanish_ladies)-1):
    if check_line_intersect(coastline_data, [[spanish_ladies[i]["lon"], spanish_ladies[i]["lat"]], [spanish_ladies[i+1]["lon"], spanish_ladies[i+1]["lat"]]]):
        folium.PolyLine(
            locations=[[spanish_ladies[i]["lat"], spanish_ladies[i]["lon"]], [spanish_ladies[i+1]["lat"], spanish_ladies[i+1]["lon"]]],
            color='red'
        ).add_to(uk_south_coast_showing_intersections)
    else:
        folium.PolyLine(
            locations=[[spanish_ladies[i]["lat"], spanish_ladies[i]["lon"]], [spanish_ladies[i+1]["lat"], spanish_ladies[i+1]["lon"]]],
            color='green'
        ).add_to(uk_south_coast_showing_intersections)

## Display the map

uk_south_coast_showing_intersections

## Sighting land

In the previous notebook, we discussed dead reckoning. One of the assumptions we made was that the ship never sighted land until the end of their journey, thus the error was compounding. In reality, when they sighted land, they could make an accurate fix to their location. Here, we are going look at how we can bring that data into our map.

#### How close 

Visibility of land depends on how high you are above the water. This is known as the dipping distance, and the higher you are, the further you can see. While we have good information on some ships, and the height of their crows nest, for this notebook we are going to assume a 20 nautical mile visibility. This is a reasonable assumption for a ship of the time, and is also a nice round number.

In [ ]:
## Convert 20 nautical miles to decimal degrees

nautical_miles = 20
nautical_miles_to_degrees = 0.01666666666666666666666666666667
dipping_distance = nautical_miles_to_degrees * nautical_miles

## for each waypoint calcuate the distance from the coast

for waypoint in spanish_ladies:
    for feature in west_europe_coastline:
        for coord in feature["geometry"]["coordinates"]:
            if geodesic((waypoint["lat"], waypoint["lon"]), (coord[1], coord[0])).nm < nautical_miles:
                waypoint["distance"] = geodesic((waypoint["lat"], waypoint["lon"]), (coord[1], coord[0])).nm
                waypoint["dipping_point"] = {
                    "lat": coord[1],
                    "lon": coord[0]
                }
                break

## Create a new geojson file following the coastline, but 20 nautical miles out to sea

coastline_buffer = {
    "type": "FeatureCollection",
    "features": []
}

for feature in west_europe_coastline:
    for coord in feature["geometry"]["coordinates"]:
        coastline_buffer["features"].append({
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Point",
                "coordinates": [coord[0] + dipping_distance, coord[1]]
            }
        })

for feature in west_europe_coastline:
    for coord in feature["geometry"]["coordinates"]:
        coastline_buffer["features"].append({
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Point",
                "coordinates": [coord[0] - dipping_distance, coord[1]]
            }
        })

for feature in west_europe_coastline:
    for coord in feature["geometry"]["coordinates"]:
        coastline_buffer["features"].append({
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Point",
                "coordinates": [coord[0], coord[1] + dipping_distance]
            }
        })

for feature in west_europe_coastline:
    for coord in feature["geometry"]["coordinates"]:
        coastline_buffer["features"].append({
            "type": "Feature",
            "properties": {},
            "geometry": {
                "type": "Point",
                "coordinates": [coord[0], coord[1] - dipping_distance]
            }
        })

## Create a map of the south coast of the UK, with some enough padding to show Northern France and the Isles of Scilly and additional room.

uk_south_coast = folium.Map(
    location=[50.5, -1.5],
    zoom_start=7,
    tiles='Stamen Terrain'
)

## Add the western european coastline to the map

folium.GeoJson(
    coastline_buffer,
    name='geojson'
).add_to(uk_south_coast)

## Display the map

uk_south_coast

## The weather

Another set of data avaible to us is weather. This can often be found as part of the ships log, but there is also historic weather data avaiable through Discovery. Unfortunatly, the historic data avaiable through Discovery is not thoroughly digitised yet, so here we are going to create some imaginary weather data. If the logs you are using have weather data, try using that instead.

Folium has some of the icons we want when recording the data - we are going to replace the waypoint markers with (boat markers)[https://python-visualization.github.io/folium/latest/user_guide/plugins/boat_marker.html]. This will provide us with a visualisation for the ships heading, as well as the wind direction and speed.

We will start by creating a new set of waypoint information. This will use the same waypoint data from before, but we're going to add in the wind and heading at each waypoint. We are going to assume that the wind is blowing from the west and backing over the journey to the south - the song mentions they "hove our ship to, with the wind from sou'west, boys," so having it wonder from a westerly to a southerly is roughly accurate and provides some variation. We're also going to assume its a force 4 and variable over the journey.

If you have weather data, you can change the data in this cell to use that instead, or adapt the code here and shown in the previous notebook when asking for bearing speed and time to add your own weather data.

Here we be using geopy to calculate the bearing, but as per [this link], geopy does not support this yet. For the perpose of this notebook and as practice we are going to make our own function to do this. As, when following a great-circle route, your bearing constanly changes to follow a straight line, we are going to assume a constant bearing over the journey instead. 

In [ ]:
## creating the constant bearing angle between two points function

def constant_bearing_angle(point1, point2):
    if point1[0] == point2[0]:
        if point1[1] > point2[1]:
            return 270
        else:
            return 90
    else:
        angle = math.degrees(math.atan((point1[1] - point2[1]) / (point1[0] - point2[0])))
        if point1[0] > point2[0]:
            return angle + 180
        else:
            return angle

## The new dictionary, empty for now

waypoints_with_weather = []

## Loop through the spanish ladies waypoints and add them to the new dictionary
## We need to add the following information:
## - heading: the boats heading in degrees. We will assume this is directly towards the next waypoint (which we can calculate using the geopy library, as we did in the previous notebook)
## - wind_direction: the direction of the wind in degrees. As discussed, we are going to assume a gradual change from a westerly to a southerly. 
## - wind_speed: the speed of the wind in knots. This is where we add our variable force 4 wind. Note that a force 4 in knots is 11-16 knots.
## Note that we are using our constant bearing angle function to calculate the heading

for i in range(len(spanish_ladies)-1):
    waypoints_with_weather.append({
        "current_name": spanish_ladies[i]["current_name"],
        "song_name": spanish_ladies[i]["song_name"],
        "lat": spanish_ladies[i]["lat"],
        "lon": spanish_ladies[i]["lon"],
        #"heading": geodesic((spanish_ladies[i]["lat"], spanish_ladies[i]["lon"]), (spanish_ladies[i+1]["lat"], spanish_ladies[i+1]["lon"])).bearing,
        "heading": constant_bearing_angle((spanish_ladies[i]["lat"], spanish_ladies[i]["lon"]), (spanish_ladies[i+1]["lat"], spanish_ladies[i+1]["lon"])),
        "wind_direction": 270 - (i * 22.5),
        "wind_speed": 11 + (i * 2.5)
    })

## Add the last waypoint

waypoints_with_weather.append({
    "current_name": spanish_ladies[-1]["current_name"],
    "song_name": spanish_ladies[-1]["song_name"],
    "lat": spanish_ladies[-1]["lat"],
    "lon": spanish_ladies[-1]["lon"],
    "heading": 0,
    "wind_direction": 270 - (len(spanish_ladies) * 22.5),
    "wind_speed": 11 + (len(spanish_ladies) * 2.5)
})

And now, like before, we add the new waypoints to the map. We also need to ensure we add the boat marker and wind information using the BoatMarker plugin. We can see the documentation for this [here](https://python-visualization.github.io/folium/plugins.html#folium.plugins.BoatMarker).

In [ ]:
## New map with the waypoints and the coastline

uk_south_coast_with_weather = folium.Map(
    location=[50.5, -1.5],
    zoom_start=7,
    tiles='Stamen Terrain'
)

## Add the western european coastline to the map

folium.GeoJson(
    coastline_data,
    name='geojson'
).add_to(uk_south_coast_with_weather)

## Add the waypoints to the map

for waypoint in waypoints_with_weather:
    folium.Marker(
        location=[waypoint["lat"], waypoint["lon"]],
        popup=waypoint["current_name"],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(uk_south_coast_with_weather)

## Add the boat markers and wind arrows to the map

for waypoint in waypoints_with_weather:
    folium.plugins.BoatMarker(
        location=[waypoint["lat"], waypoint["lon"]],
        heading=waypoint["heading"],
        wind_heading=waypoint["wind_direction"],
        wind_speed=waypoint["wind_speed"],
        color="#FF0000",
        icon_url="https://cdn0.iconfinder.com/data/icons/transportation-2-6/66/Boat-512.png"
    ).add_to(uk_south_coast_with_weather)

## And add the path, using the same technique as before to check whether the line intersects with the coastline

for i in range(len(spanish_ladies)-1):
    if check_line_intersect(coastline_data, [[spanish_ladies[i]["lon"], spanish_ladies[i]["lat"]], [spanish_ladies[i+1]["lon"], spanish_ladies[i+1]["lat"]]]):
        folium.PolyLine(
            locations=[[spanish_ladies[i]["lat"], spanish_ladies[i]["lon"]], [spanish_ladies[i+1]["lat"], spanish_ladies[i+1]["lon"]]],
            color='red'
        ).add_to(uk_south_coast_with_weather)
    else:
        folium.PolyLine(
            locations=[[spanish_ladies[i]["lat"], spanish_ladies[i]["lon"]], [spanish_ladies[i+1]["lat"], spanish_ladies[i+1]["lon"]]],
            color='green'
        ).add_to(uk_south_coast_with_weather)

## Display the map

uk_south_coast_with_weather

### Other weather

We can also add other weather data to the map. There is no weather map plugin, so we have to be a little more creative here. We are going to show a couple of creative options, but there are going to be many ways to do things - experiment and see what works for you! 

#### Effet on an area

If we know of a storm in the area, and roughly the area it covers, we can show this on the map. There are a couple of options for this, and we are going to look at both.

###### Option 1: Circle

The first option is to use a semicircle. This is the simplest option, and we can see the documentation for this [here](https://python-visualization.github.io/folium/modules.html#folium.vector_layers.CircleMarker).

This option is simpler in both code and data requrements - all we need as a centre point and a radius. 

###### Option 2: Heatmap

The second option is to use a heatmap. This is a bit more complicated, but it gives us a better idea of the area the storm covers. We can see the documentation for this [here](https://python-visualization.github.io/folium/modules.html#folium.plugins.HeatMap).

This option needs much more data about the storm itself; but, if we have this data, we can map the area the storm covers much more accurately, and include information about the strength of the storm. Another advantage of the heatmap is, if we have animated the map, we can do the same with the heatmap and show how the storm moves over time.

#### Implementing them here

We are going to implement both of these options here to demonstrate how they work. We are going to use artificial data for both, and stick them to the map with ship markers on - the aim being to show how layering more data can add more information, and start to indicate that adding too many layers can make the map hard to read.

In [ ]:
## Random radius and location in the western channel

import random
import numpy

random_radius = random.randint(0, 20)
random_lat = random.uniform(49.0, 50.5)
random_lon = random.uniform(-6.0, -2.0)

## Random generation of a dense storm at a random location in the western channel

heat_map_storm_data = (
    numpy.random.normal(size=(10000, 2)) *
    numpy.array([[0.01, 0.01]]) +
    numpy.array([[random_lat, random_lon]])
)


## add the heatmap to the map

folium.plugins.HeatMap(
    heat_map_storm_data,
    radius=15,
    blur=10,
    min_opacity=0.5
).add_to(uk_south_coast_with_weather)

## adding the circle storm to the map

folium.Circle(
    location=[random_lat, random_lon],
    radius=random_radius * 1000,
    color='red',
    fill=True,
    fill_color='red'
).add_to(uk_south_coast_with_weather)

## Display the map

uk_south_coast_with_weather

## And now, over to you

Here we're just showing an example on how to highlight 